In [154]:
import sys

sys.path.append("../src")

In [155]:
import numpy as np
import matplotlib.pyplot as plt
import importlib
from scipy.integrate import quad
from scipy.optimize import minimize
from scipy.linalg import eigh
import sto_ng
import sto_ng_overlap_function
import h5py
import gaussian_overlap_function


importlib.reload(sto_ng)
importlib.reload(sto_ng_overlap_function)
importlib.reload(gaussian_overlap_function)
from sto_ng import *
from sto_ng_overlap_function import *
from gaussian_overlap_function import *

In [156]:
with h5py.File("sto_ng.h5", "r") as f:
    alphas = f["1s/sto-3g/alpha"][:]
    cs = f["1s/sto-3g/c"][:]
alphas = np.array(alphas)
cs = np.array(cs)
cs_new = cs * (2 * alphas / np.pi) ** (3 / 4)

In [157]:
def get_smat(cs, alphabetas, Rs):
    s_mat = np.zeros((2, 2))
    for i in range(2):
        for j in range(2):
            s_mat[i, j] = Sto3g.norm(cs, alphabetas[i], alphabetas[j], Rs[i], Rs[j])
    return s_mat

In [158]:
def get_kinetic_mat(cs, alphabetas, Rs):
    kinetic_mat = np.zeros((2, 2))
    for i in range(2):
        for j in range(2):
            kinetic_mat[i, j] = Sto3g.kinetic(
                cs, alphabetas[i], alphabetas[j], Rs[i], Rs[j]
            )
    return kinetic_mat

In [159]:
def get_potential_mat_0(cs, alphabetas, Rs):
    potential_mat = np.zeros((2, 2))
    for i in range(2):
        for j in range(2):
            potential_mat[i, j] = 2 * Sto3g.potential(
                cs, alphabetas[i], alphabetas[j], Rs[i], Rs[j], Rs[0]
            )
    return potential_mat


def get_potential_mat_1(cs, alphabetas, Rs):
    potential_mat = np.zeros((2, 2))
    for i in range(2):
        for j in range(2):
            potential_mat[i, j] = Sto3g.potential(
                cs, alphabetas[i], alphabetas[j], Rs[i], Rs[j], Rs[1]
            )
    return potential_mat


def get_potential_mat(cs, alphabetas, Rs):
    return get_potential_mat_0(cs, alphabetas, Rs) + get_potential_mat_1(
        cs, alphabetas, Rs
    )

In [160]:
def get_ee_coulomb_mat(cs, alphabetas, Rs):
    ee_coulomb_mat = np.zeros((2, 2, 2, 2))
    for i in range(2):
        for j in range(2):
            for k in range(2):
                for l in range(2):
                    ee_coulomb_mat[i, j, k, l] = Sto3g.ee_coulomb(
                        cs,
                        alphabetas[i],
                        alphabetas[j],
                        alphabetas[k],
                        alphabetas[l],
                        Rs[i],
                        Rs[j],
                        Rs[k],
                        Rs[l],
                    )
    return ee_coulomb_mat


c_chi = np.array([1, 1])
c_chi @ get_ee_coulomb_mat(cs, alphabetas, Rs) @ c_chi

array([[ 20.74788356,  41.21098099],
       [ 41.21098099, 107.58155944]])

In [161]:
def get_1e_mat(cs, alphabetas, Rs):
    kinetic_mat = get_kinetic_mat(cs, alphabetas, Rs)
    potential_mat = get_potential_mat(cs, alphabetas, Rs)
    return kinetic_mat + potential_mat


def get_2e_mat(c_chi, cs, alphabetas, Rs):
    ee_coulomb_mat = get_ee_coulomb_mat(cs, alphabetas, Rs)
    return c_chi @ ee_coulomb_mat @ c_chi


def get_fock_mat(c_chi, cs, alphabetas, Rs):
    kinetic_mat = get_kinetic_mat(cs, alphabetas, Rs)
    potential_mat = get_potential_mat(cs, alphabetas, Rs)
    ee_coulomb_mat = c_chi @ get_ee_coulomb_mat(cs, alphabetas, Rs) @ c_chi
    fock_mat = kinetic_mat + potential_mat + ee_coulomb_mat
    return fock_mat

In [162]:
zetaHe = 2.0925
zetaH = 1.24
alphabetas = [alphas * zetaHe**2, alphas * zetaH**2]
Rs = [np.array([0, 0, 0]), np.array([0, 0, 1.4632])]
c_chi = np.array([1, 0.1])
for i in range(100):
    s_mat = get_smat(cs_new, alphabetas, Rs)
    fock_mat = get_fock_mat(c_chi, cs_new, alphabetas, Rs)
    val, vec = eigh(fock_mat, s_mat)
    c_chi = 0.5 * c_chi + 0.5 * vec[:, 0]
    print(
        val[0] * 2
        - vec[:, 0]
        @ (vec[:, 0] @ get_ee_coulomb_mat(cs_new, alphabetas, Rs) @ vec[:, 0])
        @ vec[:, 0]
    )
c_chi
s_mat = get_smat(cs, alphabetas, Rs)
s_mat

-6.21560129522436
-6.298714603815002
-5.438081123987764
-4.821918295635054
-4.5282569355639986
-4.385675442070957
-4.306741809836334
-4.267839167677576
-4.247553952623891
-4.237673998237873
-4.232559026048069
-4.230084765340621
-4.228800498769577
-4.2281832019007775
-4.227860954795278
-4.227707164469928
-4.227626281845904
-4.227588001834613
-4.227567688255461
-4.22755816828694
-4.2275530626085125
-4.227550697401321
-4.227549412948412
-4.227548826005633
-4.227548502525796
-4.227548357074623
-4.227548275507382
-4.227548239523072
-4.227548218925817
-4.227548210041256
-4.22754820483141
-4.227548202643128
-4.227548201322833
-4.22754820078544
-4.227548200450113
-4.227548200318617
-4.227548200233236
-4.227548200201204
-4.2275482001794025
-4.227548200171642
-4.227548200166056
-4.227548200164188
-4.227548200162754
-4.227548200162307
-4.227548200161937
-4.227548200161831
-4.227548200161737
-4.227548200161712
-4.227548200161686
-4.2275482001616815
-4.227548200161674
-4.227548200161675
-4.22754820

array([[ 2.2352247 ,  2.79292795],
       [ 2.79292795, 10.74119331]])

In [163]:
vec

array([[-2.42732989,  2.36781597],
       [-0.46505623, -1.4753095 ]])

In [164]:
val

array([-1.59745951, -1.1370558 ])

In [177]:
np.outer(c_chi, c_chi)

array([[5.89193041, 1.12884489],
       [1.12884489, 0.2162773 ]])

In [170]:
s_mat

array([[ 2.2352247 ,  2.79292795],
       [ 2.79292795, 10.74119331]])

In [174]:
c_chi[0] ** 2 / s_mat[0, 0]

2.635945461732266